In [53]:
import arcpy

In [54]:
arcpy.env.workspace = r"H:\UEP235_AdvGIS\labs\lab4-automation\ModelbuilderLab\ModelbuilderLab.gdb"

In [55]:
# join Florida blockgroups with ACS
florida_blockgroups_withACSTable = arcpy.management.AddJoin(
    in_layer_or_view="florida_blockgroups", in_field="GEOFIPS",
    join_table="miami_ACS_blockgroup_2018", join_field="Geo_FIPS",
    join_type="KEEP_COMMON", index_join_fields="INDEX_JOIN_FIELDS")[0]

In [56]:
# get categories for HurricaneEvac_Zone_proj 
categories = sorted({row[0] for row in arcpy.da.SearchCursor("HurricaneEvac_Zone_proj", ["CATEGORY"])})
categories

[0, 1, 2, 3, 4, 5]

In [1]:
# iterate thru categories and select features
for cat in categories:
    print(f"Working on category: {cat}")

    # create file for selection
    selection_feature = f"HurricaneEvac_Category{cat}"
    arcpy.analysis.Select(
        in_features="HurricaneEvac_Zone_proj",
        out_feature_class=selection_feature,
        where_clause=f"CATEGORY = {cat}")
    
    # pairwise intersect selected with florida_blockgroups_withACSTable
    intersect_feature = f"florida_bg_Category{cat}_intersect"
    arcpy.analysis.PairwiseIntersect(
        in_features=[selection_feature, florida_blockgroups_withACSTable],
        out_feature_class=intersect_feature)

    # pairwise dissolve
    dissolve_feature = f"florida_bg_Category{cat}_dissolve"
    arcpy.analysis.PairwiseDissolve(
        in_features=intersect_feature,
        out_feature_class=dissolve_feature,
        dissolve_field=["florida_blockgroups_GEOID"],
        statistics_fields=[
            ["miami_ACS_blockgroup_2018_SE_A14006_001", "FIRST"],
            ["miami_ACS_blockgroup_2018_SE_A10036_001", "FIRST"],
            ["miami_ACS_blockgroup_2018_SE_A18009_001", "FIRST"]])

    # summary stats
    stats_table = f"florida_bg_Category{cat}_statistics"
    arcpy.analysis.Statistics(
        in_table=dissolve_feature,
        out_table=stats_table,
        statistics_fields=[
            ["FIRST_miami_ACS_blockgroup_2018_SE_A14006_001", "MEDIAN"],
            ["FIRST_miami_ACS_blockgroup_2018_SE_A10036_001", "MEDIAN"],
            ["FIRST_miami_ACS_blockgroup_2018_SE_A14006_001", "MEAN"],
            ["FIRST_miami_ACS_blockgroup_2018_SE_A10036_001", "MEAN"]])

NameError: name 'categories' is not defined